## Predspracovanie dát a ETL

In [1]:
# dependencies
import sys
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

In [2]:
# local modules
sys.path.append("..")

from _ import constants
from _.functions import drop

### Načítanie dát

In [3]:
# train dataset
if not constants.DROPPED.exists():
    # drop redundant data if not exists
    df_train = drop(constants.TRAIN);
    df_train.to_parquet(constants.DROPPED, index=False)
else:
    df_train = pd.read_parquet(constants.DROPPED)

In [4]:
df_train.tail(5)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
9379455,ZYNMLE3MV3LK,62728015bec05,1541544490,15,interaction item image,6617798,PT,"Paris, France",desktop,None,None,None
9379456,ZYNMLE3MV3LK,62728015bec05,1541544491,16,clickout item,6617798,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...
9379457,ZYNMLE3MV3LK,62728015bec05,1541544540,17,clickout item,2712342,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...
9379458,ZYNMLE3MV3LK,62728015bec05,1541544967,18,change of sort order,interaction sort button,PT,"Paris, France",desktop,None,None,None
9379459,ZYNMLE3MV3LK,62728015bec05,1541544973,19,clickout item,1161323,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...


In [5]:
# meta dataset
df_meta = pd.read_csv(constants.METADATA)

In [6]:
df_meta.head(5)

,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


### Úprava multi-value atribútov

Rozdelenie "|" separovaných hodnôt na list.

In [8]:
df_train['impressions'] = df_train['impressions'].apply(lambda x: x.split('|') if x != None else x)

In [9]:
df_train['prices'] = df_train['prices'].apply(lambda x: list(map(int, x.split('|'))) if x != None else x)

In [10]:
df_train['current_filters'] = df_train['current_filters'].apply(lambda x: x.split('|') if x != None else x)

### Informácie o cene

Používateľ sa môže rozhodovať aj na základe ceny impresie, preto by bolo dobré získať cenu danej položky z clickoutu.
Nakoľko poradie je pri odporúčaní dôležité, spolu s cenou si uložíme aj index danej položky medzi impresiami.

In [11]:
def get_impression_index(reference,impressions):
    if impressions is not None:
        return next((index for index,impr in enumerate(impressions) if impr == reference),np.nan)
    return np.nan

In [12]:
df_train['impression_index'] = df_train.apply(lambda x: get_impression_index(x['reference'], x['impressions']), axis=1)

In [13]:
df_train['price'] = df_train.apply(lambda x: x['prices'][int(x['impression_index'])] if ~np.isnan(x['impression_index']) else np.nan, axis=1)

In [14]:
df_train['mean_price'] = df_train['prices'].apply(lambda x: np.mean(x) if x is not None else np.nan)

In [15]:
df_train

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,impression_index,price,mean_price
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,None,None,None,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,None,None,None,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,88WGXB3ZZ2HD,5ab59bd1c67df,1541173398,32,interaction item image,788211,PL,"Kolobrzeg , Poland",mobile,None,None,None,NaN,NaN,NaN
9996,88WGXB3ZZ2HD,5ab59bd1c67df,1541173398,33,interaction item image,788211,PL,"Kolobrzeg , Poland",mobile,None,None,None,NaN,NaN,NaN
9997,88WGXB3ZZ2HD,5ab59bd1c67df,1541173403,34,clickout item,788211,PL,"Kolobrzeg , Poland",mobile,None,"[788211, 1246333, 2049991, 1217228, 1393804, 2...","[65, 73, 35, 108, 79, 76, 88, 35, 54, 55, 83, ...",0.0,65.0,65.4
9998,88WGXB3ZZ2HD,5ab59bd1c67df,1541173409,35,interaction item image,788211,PL,"Kolobrzeg , Poland",mobile,None,None,None,NaN,NaN,NaN


## Enkódovanie 

### Enkódovanie kategorických atribútov

In [16]:
categorical_attributes = ['session_id', 'action_type', 'platform', 'city']

In [17]:
for cat in categorical_attributes:
    df_train[cat] = df_train[cat].astype('category')
    df_train[cat] = df_train[cat].cat.codes

In [18]:
df_train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,impression_index,price,mean_price
0,00RL8Z82B2Z1,525,1541037460,1,9,Newtown,3,539,mobile,None,None,None,NaN,NaN,NaN
1,00RL8Z82B2Z1,525,1541037522,2,4,666856,3,539,mobile,None,None,None,NaN,NaN,NaN
2,00RL8Z82B2Z1,525,1541037522,3,4,666856,3,539,mobile,None,None,None,NaN,NaN,NaN
3,00RL8Z82B2Z1,525,1541037532,4,4,666856,3,539,mobile,None,None,None,NaN,NaN,NaN
4,00RL8Z82B2Z1,525,1541037532,5,4,109038,3,539,mobile,None,None,None,NaN,NaN,NaN


### One-hot enkódovanie

Zamýšlali sme sa nad one-hot enkódovaním aj atribútu platform, ale kedže nie je z nášho pohľadu až tak dôležitý pre predikciu a obsahuje 55 rozličných hodnôt, rozhodli sme sa len pre city.

In [19]:
df_train = pd.get_dummies(df_train, columns=["device"])

In [20]:
df_train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,current_filters,impressions,prices,impression_index,price,mean_price,device_desktop,device_mobile,device_tablet
0,00RL8Z82B2Z1,525,1541037460,1,9,Newtown,3,539,None,None,None,NaN,NaN,NaN,0,1,0
1,00RL8Z82B2Z1,525,1541037522,2,4,666856,3,539,None,None,None,NaN,NaN,NaN,0,1,0
2,00RL8Z82B2Z1,525,1541037522,3,4,666856,3,539,None,None,None,NaN,NaN,NaN,0,1,0
3,00RL8Z82B2Z1,525,1541037532,4,4,666856,3,539,None,None,None,NaN,NaN,NaN,0,1,0
4,00RL8Z82B2Z1,525,1541037532,5,4,109038,3,539,None,None,None,NaN,NaN,NaN,0,1,0


## Predspracovanie metadát

In [21]:
df_meta.head(5)

,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


In [22]:
df_meta.set_index('item_id',inplace=True)

Rozdelenie "|" separovaných hodnôt na list.

In [23]:
df_meta['properties'] = df_meta['properties'].apply(lambda x: x.split('|'))

Vyparsovanie počtu hviezdičiek

In [24]:
def parse_stars(string):
    result = re.search(r'^(\d) star$',string, flags=re.IGNORECASE)
    if result != None:
        return int(result.group(1))
    
df_meta['stars'] = df_meta['properties'].apply(lambda props: next((i for i in list(map(parse_stars, props)) if i is not None), np.nan))

Vyparsovanie hodnotenia

In [25]:
rating_map = {
    'Satisfactory Rating': 7.0,
    'Good Rating': 7.5,
    'Very Good Rating': 8.0,
    'Excellent Rating': 8.5
}

    
for rating_key in rating_map:
    df_meta.loc[df_meta['properties'].apply(lambda x: rating_key in x),'rating'] = rating_map[rating_key]

In [26]:
df_meta.head(5)

,properties,stars,rating
item_id,,,
5101,"[Satellite TV, Golf Course, Airport Shuttle, C...",4.0,7.5
5416,"[Satellite TV, Cosmetic Mirror, Safe (Hotel), ...",4.0,8.5
5834,"[Satellite TV, Cosmetic Mirror, Safe (Hotel), ...",3.0,8.0
5910,"[Satellite TV, Sailing, Cosmetic Mirror, Telep...",4.0,7.5
6066,"[Satellite TV, Sailing, Diving, Cosmetic Mirro...",4.0,7.5


## Zlúčenie metadát s datasetom

In [27]:
for index,_ in enumerate(tqdm(df_train.itertuples(), desc="Iterating rows", total=df_train.shape[0])):
    row = df_train.iloc[int(index)]
    try:
        item = df_meta.loc[int(row['reference'])]
        row['stars'] = item['stars']
        row['rating'] = item['rating']
    except:
        row['stars'] = np.nan
        row['rating'] = np.nan

Iterating rows:   0%|                                                     | 0/10000 [00:00<?, ?it/s]/Users/feri/.local/share/virtualenvs/DP-WWU-SXFb/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/feri/.local/share/virtualenvs/DP-WWU-SXFb/lib/python3.8/site-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)
Iterating rows: 100%|████████████████████████████████████████| 10000/10000 [00:10<00:00, 947.56it/s]


## Výstup

In [28]:
df_train.tail(5)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,current_filters,impressions,prices,impression_index,price,mean_price,device_desktop,device_mobile,device_tablet
9995,88WGXB3ZZ2HD,253,1541173398,32,4,788211,35,274,None,None,None,NaN,NaN,NaN,0,1,0
9996,88WGXB3ZZ2HD,253,1541173398,33,4,788211,35,274,None,None,None,NaN,NaN,NaN,0,1,0
9997,88WGXB3ZZ2HD,253,1541173403,34,1,788211,35,274,None,"[788211, 1246333, 2049991, 1217228, 1393804, 2...","[65, 73, 35, 108, 79, 76, 88, 35, 54, 55, 83, ...",0.0,65.0,65.4,0,1,0
9998,88WGXB3ZZ2HD,253,1541173409,35,4,788211,35,274,None,None,None,NaN,NaN,NaN,0,1,0
9999,88WGXB3ZZ2HD,253,1541173409,36,4,788211,35,274,None,None,None,NaN,NaN,NaN,0,1,0


In [29]:
df_train.to_parquet(constants.PREPROCESSED, index=False)